**Nome:** Miguel Santos de Araujo do Nascimento

# Imports

In [ ]:
# Remover acentuação de palavras
!pip install unidecode

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

import logging

import xml.etree.ElementTree as ET

from enum import Enum

from unidecode import unidecode # Remover acentuação de palavras

# Baixando arquivos de dados

In [ ]:
!wget https://github.com/miguelsantosan/BMT_2025.2_Trabalho_1/raw/main/CysticFibrosis2.zip

In [ ]:
!unzip -o CysticFibrosis2.zip -d CysticFibrosis2

# Configurações

In [ ]:
# @title Logging e pré processamento

stop_en = stopwords.words('english')

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

file_handler = logging.FileHandler('Trab1.log')
file_handler.setLevel(logging.INFO)

formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
formatter.default_msec_format = '%s.%03d' # Mostrar milissegundos com ponto invés de vírgula
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

In [ ]:
# @title Enum LEIA e ESCREVA

"""
Pra poder passar pras funções
Assim não fico passando magic numbers
"""
# LEIA = 1
# ESCREVA = 2

class cfgEnum(Enum):
  LEIA = 1
  ESCREVA = 2

In [ ]:
# @title Arquivos .dtd

PROCESSADOR_CONSULTAS_FORMATO = r'CysticFibrosis2/data/cfc2-query.dtd'
GLI_FORMATO = r'CysticFibrosis2/data/cfc2.dtd'

## Criando arquivos .CFG

In [ ]:
# @title PC.CFG

CONFIG_PC_CONTENT = """LEIA=CysticFibrosis2/data/cfquery.xml
CONSULTAS=consultas.csv
ESPERADOS=esperados.csv
"""

CONFIG_PC_NAME = "PC.CFG"

with open(CONFIG_PC_NAME, "w") as f:
    f.write(CONFIG_PC_CONTENT)

print(f"'{CONFIG_PC_NAME}' created successfully.")

In [ ]:
# @title GLI.CFG

CONFIG_GLI_CONTENT = """LEIA=CysticFibrosis2/data/cf74.xml
LEIA=CysticFibrosis2/data/cf75.xml
LEIA=CysticFibrosis2/data/cf76.xml
LEIA=CysticFibrosis2/data/cf77.xml
LEIA=CysticFibrosis2/data/cf78.xml
LEIA=CysticFibrosis2/data/cf79.xml
ESCREVA=lista_invertida.csv
"""

CONFIG_GLI_NAME = "GLI.CFG"

with open(CONFIG_GLI_NAME, "w") as f:
    f.write(CONFIG_GLI_CONTENT)

print(f"'{CONFIG_GLI_NAME}' created successfully.")

In [ ]:
# @title INDEX.CFG

CONFIG_INDEX_CONTENT = """LEIA=lista_invertida.csv
ESCREVA=modelo.csv
"""

CONFIG_INDEX_NAME = "INDEX.CFG"

with open(CONFIG_INDEX_NAME, "w") as f:
    f.write(CONFIG_INDEX_CONTENT)

print(f"'{CONFIG_INDEX_NAME}' created successfully.")

In [ ]:
# @title BUSCA.CFG

CONFIG_BUSCA_CONTENT = """MODELO=modelo.csv
CONSULTAS=consultas.csv
RESULTADOS=resultado.csv
"""

CONFIG_BUSCA_NAME = "BUSCA.CFG"

with open(CONFIG_BUSCA_NAME, "w") as f:
    f.write(CONFIG_BUSCA_CONTENT)

print(f"'{CONFIG_BUSCA_NAME}' created successfully.")

In [ ]:
# @title (DELETAR) Teste
# logger.debug("This is a debug message.")
# logger.info("This is an info message.")
# logger.warning("This is a warning message.")
# logger.error("This is an error message.")
# logger.critical("This is a critical message.")

# Funções

In [ ]:
# @title Logging - calcular tempo

import time

def CalculaTempoMedio(duracoes):
  tempo_medio = sum(duracoes)/len(duracoes)
  return tempo_medio

In [ ]:
# @title Pegando arquivos de uma pasta

import glob
import os

def RetornaNomeArquivoDoPath(file_path):
  """
  Retorna apenas o nome do arquivo a partir do path
  """
  # Faz split no path, vira uma lista, pega o último elemento
  file_name = file_path.split('/')[-1]
  return file_name

def RetornaArquivosNoDiretorio_old(dir_path, show_files=False):
  """
  Essa função precisa do path completo da pasta,
  então vou usar a outra que percorre as subpastas
  """
  file_paths = glob.glob(dir_path + '/*')

  if show_files:
    print(f'Peguei os arquivos da pasta {dir_path}')
    for file_path in file_paths:
      print(file_path)

  return file_paths

def RetornaArquivosNoDiretorio(dir_path, file_type=None, ignore_list=None, show_files=False):
  """
  Retorna os arquivos de dir_path, percorrendo as subpastas
  file_type == None --> todos os arquivos
  file_type != None --> apenas arquivos .file_type
  ignore_list != None --> ignoro os arquivos na lista
  """
  file_paths = []
  #ignore_list = ['cfquery.xml']

  if file_type == None:
    logger.info(f'Buscando todos os arquivos da pasta {dir_path} para armazenar em memória')
  else:
    logger.info(f'Buscando todos os arquivos .{file_type} da pasta {dir_path} para armazenar em memória')

  for root, _, files in os.walk(dir_path):
      for file in files:
        # Se tiver arquivos para ignorar, verifica se o arquivo atual está em ignore_list
        if ignore_list != None:
          file_path = os.path.join(root, file)
          file_name = RetornaNomeArquivoDoPath(file_path)
          if file_name in ignore_list:
            continue

        if file_type == None: # Todos os arquivos
          file_paths.append(os.path.join(root, file))
        else: # Apenas arquivos de um tipo específico
          if (file_type != None) and (file.endswith(f'.{file_type}')):
              file_paths.append(os.path.join(root, file))

  if file_type == None:
    logger.info(f'Todos os arquivos da pasta {dir_path} armazenados em memória com sucesso')
  else:
    logger.info(f'Arquivos .{file_type} da pasta {dir_path} armazenados em memória com sucesso')

  if show_files:
    #print(f'Peguei todos os arquivos .{file_type} da pasta {dir_path}')
    for file_path in file_paths:
      print(file_path)

  return file_paths

In [ ]:
# @title Pré-processamento do texto recuperado
from unidecode import unidecode

def TrocaNewlinePorEspaco(texto):
  """
  Substitui caracteres de nova linha (\n) por espaços em uma string.
  """
  if isinstance(texto, str):
    return texto.replace('\n', ' ')
  else:
    logger.error('Não foi passada uma string para a função TrocaNewlinePorEspaco()')
    return texto

def RemoveAspas(texto):
  """
  Remove aspas (simples e duplas) de uma string.
  """
  if isinstance(texto, str):
    texto = texto.replace("'", "")
    texto = texto.replace("\"", "")
    return texto
  else:
    logger.error('Não foi passada uma string para a função RemoveAspas()')
    return texto

def RemovePontuacao(texto):
  """
  Remove aspas (simples e duplas) de uma string.
  """
  if isinstance(texto, str):
    texto = texto.translate(str.maketrans('', '', '.,;!?()[]{}<>=´`~^')) # Remove todos esses caracteres, melhor que ter vários replace
    return texto
  else:
    logger.error('Não foi passada uma string para a função RemoveAspas()')
    return texto

def RemoveAcentuacao(texto):
  if isinstance(texto, str):
    return unidecode(texto)
  else:
    logger.error('Não foi passada uma string para a função RemoveAcentuacao()')
    return texto

def RemoveStopWords(texto):
  """
  Remove as stop words de uma string.
  """
  if isinstance(texto, str):
    tokens = word_tokenize(texto)
    tokens_filtrados = [p for p in tokens if p.lower() not in stop_en]
    return ' '.join(tokens_filtrados)
  else:
    logger.error('Não foi passada uma string para a função RemoveStopWords()')
    return texto

def PreProcessamentoDeTextoRecuperado(texto, remove_stopwords=False):
  texto = TrocaNewlinePorEspaco(texto)
  texto = RemoveAspas(texto)
  texto = RemovePontuacao(texto)
  texto = RemoveAcentuacao(texto)
  if remove_stopwords:
    texto = RemoveStopWords(texto)
  return texto

In [ ]:
# @title (DELETAR) Testando pré-processamento de texto
t = "STÈÉL ~ÎRÔN METÃL 'asa\/\/\/d\n\n[{(()}}{]be... bebe ;;....,,,.;be be be is i am 'asda'fgrttaaa\"\""
print(t)
#t=t.replace("'", "")
#t=t.replace("\"", "")
t = PreProcessamentoDeTextoRecuperado(t)
print(t)
print(type(t))
print(len(t))

t = "Cléber François Úl`[´]`làáíìúùòóèé-asbb"
t = RemovePontuacao(t)
t = unidecode(t)
t

In [ ]:
# @title (DELETAR) Testando correção de texto

test_text1 = """The significance of Pseudomonas aeruginosa infection in the
respiratory tract of 9 cystic fibrosis patients have been studied
by means of immunoelectrophoretical analysis of patients' sera for
the number of precipitins against Pseudomonas aeruginosa and the
concentrations of 16 serum proteins.  In addition, the clinical and
radiographical status of the lungs have been evaluated using 2
scoring systems.  Precipitins against Pseudomonas aeruginosa were
demonstrated in all sera, the maximum number in one serum was 22.
The concentrations of 12 of the serum proteins were significantly
changed compared with matched control persons.  Notably IgG and IgA
were elevated and the "acute phase proteins" were changed, the
latter suggesting active tissue damage.  The concentrations of 3 of
the acute phase proteins, notably haptoglobin, were correlated to
the number of precipitins suggesting that the respiratory tract
infection in patients with many precipitins is accompanied by more
tissue damage than the infection in patients with few precipitins.
The results indicate no protective value of the many precipitins on
the tissue of the respiratory tract."""

test_text2 = """Dr. Economou-Mavrou and his colleagues appear to have misunderstood
my previous note concerning skin wrinkling in cystic fibrosis.  The
range of normality varies in different climatic conditions, and in
Aukland wrinkling does not appear in cool tap-water (18-22 C) under
3 minutes in normals.  Skin wrinkling on immersion appears to be
age-related, but is difficult to detect reliably in the newborn.
Prior washing of the hands, or immersion in sea-water (even the day
before), accelerates skin wrinkling, as does warm weather.  I would
suggest that "normality" needs redefining in Athens if 6 of 17
"normals" were "abnormal"."""

test_text1 = PreProcessamentoDeTextoRecuperado(test_text1)
test_text2 = PreProcessamentoDeTextoRecuperado(test_text2)

test_tokens1 = word_tokenize(test_text1)
test_tokens2 = word_tokenize(test_text2)

print(test_tokens1)
print(test_tokens2)

# Armazenando arquivos

In [ ]:
# @title Arquivos que não serão adicionados à base

ignore_list = ['cfquery.xml']

In [ ]:
dir_path = 'CysticFibrosis2/'
file_paths_CysticFibrosis2 = RetornaArquivosNoDiretorio(dir_path,  'xml', ignore_list, show_files=True)

# Processador de consultas

In [ ]:
#%%writefile processador_de_consultas.py

# Gerador lista invertida

In [ ]:
# @title Funções para gerar a lista

#%%writefile gerador_lista_invertida.py
import logging, time, inspect
import xml.etree.ElementTree as ET

def NomeModulo_GeradorListaInvertida():
  """
  Se quiser mudar o nome que aparece no logger, só alterar a string retornada nessa função
  """
  return 'Gerador Lista Invertida'

def GeraListaInvertida(base):
  """
  base é uma lista de tuplas, onde o primeiro elemento é o ID do documento e o segundo é o texto do documento
  Percorre todos os textos e gera a lista invertida
  Funciona com lista de texto, assim posso ter doc: [doc, [doc_texto1, doc_texto2, ...]] e contam para o mesmo documento
  Retorna a lista invertida
  """
  nome_modulo = NomeModulo_GeradorListaInvertida()
  nome_funcao = inspect.currentframe().f_code.co_name # Retorna o nome da função pra passar no logger
  lista_invertida = {}

  logger.info(f'{nome_modulo} ({nome_funcao}) - Gerando lista invertida')

  duracoes = []
  for doc_tupla in base:
    duracoes = []
    tempo_init = time.time()
    doc_id = doc_tupla[0]
    doc_info = doc_tupla[1]

    # Se for string única, transforma em lista de 1 elemento
    if isinstance(doc_info, str):
        doc_info = [doc_info]

    for info in doc_info:
      fraselc = str.upper(info).split()
      for palavra in fraselc:
        docs = lista_invertida.get(palavra, [])
        docs.append(doc_id)
        lista_invertida[palavra]=docs

    tempo_final = time.time()
    duracao = tempo_final - tempo_init
    duracoes.append(duracao)

  tempo_medio = CalculaTempoMedio(duracoes)
  tempo_medio_ms = tempo_medio * 1000

  logger.info(f'{nome_modulo} ({nome_funcao}) - Lista invertida gerada com sucesso')
  #logger.info(f'{nome_modulo} ({nome_funcao}) - Tempo médio de processamento de um documento para gerar a lista: {tempo_medio:.5f} segundos')
  logger.info(f'{nome_modulo} ({nome_funcao}) - Tempo médio de processamento de um documento para gerar a lista: {tempo_medio_ms:.3f} milissegundos')

  return lista_invertida

def LeArquivo_GLI_CFG(flag, config_file='GLI.CFG'):
  """
  Lê o arquivo GLI.CFG
  flag == cfgEnum.LEIA --> Pego os arquivos XML que vão ser lidos
  flag == cfgEnum.ESCREVA --> Pega o nome do CSV com a lista invertida
  """
  nome_modulo = NomeModulo_GeradorListaInvertida()
  nome_funcao = inspect.currentframe().f_code.co_name # Retorna o nome da função pra passar no logger
  filename = None
  file_paths = [] # Initialize file_paths list

  try:
    with open(config_file, 'r') as f:
      match flag:
        case cfgEnum.LEIA:
          logger.info(f'{nome_modulo} ({nome_funcao}) - Lendo arquivo {config_file} para pegar os arquivos XML para ler')
          for line in f:
            line = line.strip()
            if line.startswith('LEIA='):
              path = line.split('=', 1)[1]
              file_paths.append(path)
          logger.info(f'{nome_modulo} ({nome_funcao}) - Arquivos XML para ler recuperados com sucesso')

          return file_paths

        case cfgEnum.ESCREVA:
          logger.info(f'{nome_modulo} ({nome_funcao}) - Lendo arquivo {config_file} para pegar o nome do CSV com a lista invertida')
          for line in f:
            line = line.strip()
            if line.startswith('ESCREVA='):
              filename = line.split('=', 1)[1]
              logger.info(f'{nome_modulo} ({nome_funcao}) - Nome do CSV com a lista invertida recuperado com sucesso')

              return filename

        case _:
          logger.error(f'{nome_modulo} ({nome_funcao}) - Flag inválida: \'{flag}\'. Use flag \'cfgEnum.LEIA\' ou \'cfgEnum.ESCREVA\'')
          return

  except FileNotFoundError:
    logger.error(f'{nome_modulo} ({nome_funcao}) - Arquivo {config_file} não encontrado.')
    return
  except Exception as e:
    logger.error(f"{nome_modulo} ({nome_funcao}) - Erro ao ler arquivo de configuração '{config_file}': {e}")
    return

def SalvaListainvertidaEmCsv(lista_invertida_dict, config_file='GLI.CFG', separator=';'):
  """
  Lê GLI.CFG e pega o nome do CSV para salvar a lista invertida
  """
  nome_modulo = NomeModulo_GeradorListaInvertida()
  nome_funcao = inspect.currentframe().f_code.co_name # Retorna o nome da função pra passar no logger
  filename = LeArquivo_GLI_CFG(cfgEnum.ESCREVA) # Pega o nome do CSV

  try:
    with open(filename, 'w', newline='') as csvfile:
      for key, values in lista_invertida_dict.items():
        # Pega a lista dos documentos
        values_str = '[' + ','.join(map(str, values)) + ']'
        csvfile.write(f"{key}{separator}{values_str}\n")
    logger.info(f'{nome_modulo} ({nome_funcao}) - Lista invertida salva em {filename} com sucesso')
  except IOError as e:
    logger.error(f'{nome_modulo} ({nome_funcao}) - Erro ao salvar a lista invertida em {filename}: {e}')

# Função para debug
def PrintListaInvertida(lista_invertida):
  chaves = [i for i in lista_invertida.keys() ]
  chaves.sort()

  for i in chaves:
    print(" ",i," & ",str(lista_invertida[i]), " \\\\")

def PegaDocumentosDosArquivosXML(file_paths):
  """
  Recebe uma lista de caminhos para arquivos XML ou um arquivo único
  Pega todos os elementos RECORD e pega o RECORDNUM de cada um deles
  Se tiver ABSTRACT, adiciona a tupla (RECORDNUM, ABSTRACT) à lista de records_data
  Se não tiver ABSTRACT, mas tiver EXTRACT, adiciona a tupla (RECORDNUM, EXTRACT) à lista de records_data
  Se não tiver ABSTRACT nem EXTRACT, desconsidera o documento

  Retorna a lista com as tuplas
  """
  nome_modulo = NomeModulo_GeradorListaInvertida()
  nome_funcao = inspect.currentframe().f_code.co_name # Retorna o nome da função pra passar no logger
  records_data = []
  records_total = 0 # Número total de documentos nos arquivos XML

  # Posso passar uma lista ou um arquivo único
  # Se for um arquivo único, converte em lista de 1 elemento
  if isinstance(file_paths, str):
    file_paths = [file_paths]

  duracoes_por_arquivo_xml = []
  duracoes_por_record = []
  for file_path in file_paths:
    tempo_arquivo_xml_init = time.time()
    duracoes_por_record = []
    records_armazenados = 0 # Records que possuem ABSTRACT ou EXTRACT
    try:
      tree = ET.parse(file_path)
      root = tree.getroot()
      record_elements = root.findall('.//RECORD') # Pego todos os elementos RECORD do arquivo
      records_total += len(record_elements)
      file_name = RetornaNomeArquivoDoPath(file_path)

      logger.info(f'{nome_modulo} ({nome_funcao}) - {file_name} possui {len(record_elements)} documentos')

      if not record_elements:
        logger.error(f'Arquivo {file_path} não possui RECORD')
        continue

      for record in record_elements:
        tempo_record_init = time.time()

        record_num_element = record.find('RECORDNUM')
        abstract_element = record.find('ABSTRACT')
        extract_element = record.find('EXTRACT')

        record_num = record_num_element.text if record_num_element is not None else None
        abstract = abstract_element.text if abstract_element is not None else None
        extract = extract_element.text if extract_element is not None else None

        if record_num is None: # Se por acaso não tiver RECORDNUM
          logger.warning(f'{nome_modulo} ({nome_funcao}) - Erro ao pegar o RECORDNUM de um dos documentos no arquivo {file_name}')
        elif abstract is not None: # Tem ABSTRACT
          abstract = PreProcessamentoDeTextoRecuperado(abstract)
          records_data.append((record_num, abstract))
          records_armazenados += 1
        elif extract is not None: # Não tem ABSTRACT, mas tem EXTRACT
          logger.warning(f'{nome_modulo} ({nome_funcao}) - Documento com RECORDNUM = {record_num} não possui ABSTRACT, vou usar o EXTRACT')
          extract = PreProcessamentoDeTextoRecuperado(extract)
          records_data.append((record_num, extract))
          records_armazenados += 1
        else: # Não tem ABSTRACT nem EXTRACT, desconsidero o documento
          logger.error(f'{nome_modulo} ({nome_funcao}) - Documento com RECORDNUM = {record_num} ({file_name}) não possui ABSTRACT nem EXTRACT, vou desconsiderar')

        tempo_record_final = time.time()
        duracao_record = tempo_record_final - tempo_record_init
        duracoes_por_record.append(duracao_record)

      tempo_medio_records_ms = CalculaTempoMedio(duracoes_por_record) * 1000
      logger.info(f'{nome_modulo} ({nome_funcao}) - Tempo médio de processamento de um RECORD do arquivo {file_name} para gerar a lista: {tempo_medio_records_ms:.3f} milissegundos')

    except FileNotFoundError:
      logger.error(f'{nome_modulo} ({nome_funcao}) - arquivo {file_path} não encontrado')
    except ET.ParseError as e:
      logger.error(f'{nome_modulo} ({nome_funcao}) - Erro ao fazer parsing no arquivo XML {file_path}: {e}')
    except Exception as e:
      logger.error(f'{nome_modulo} ({nome_funcao}) - Erro ao processar o arquivo {file_path}: {e}')

    tempo_arquivo_xml_final = time.time()
    duracao_arquivo_xml = tempo_arquivo_xml_final - tempo_arquivo_xml_init
    duracoes_por_arquivo_xml.append(duracao_arquivo_xml)

    logger.info(f'{nome_modulo} ({nome_funcao}) - Arquivo {file_name} processado com sucesso. Foram armazenados {records_armazenados}/{len(record_elements)} documentos')

  logger.info(f'{nome_modulo} ({nome_funcao}) - Foram processados {len(file_paths)} arquivos XML e armazenados {len(records_data)}/{records_total} documentos')

  tempo_medio_arquivo_xml = CalculaTempoMedio(duracoes_por_arquivo_xml)
  tempo_medio_arquivo_xml_ms = tempo_medio_arquivo_xml * 1000
  #logger.info(f'{nome_modulo} ({nome_funcao}) - Tempo médio de processamento de um arquivo XML para recuperar o texto: {tempo_medio_arquivo_xml:.3f} segundos')
  logger.info(f'{nome_modulo} ({nome_funcao}) - Tempo médio de processamento de um arquivo XML para recuperar o texto: {tempo_medio_arquivo_xml_ms:.3f} milissegundos')

  return records_data

def Modulo_GeradorListaInvertida():
  """
  Executa todos os passos para gerar a lista invertida e salvar em um arquivo CSV

  Retorna a lista invertida
  """
  nome_modulo = NomeModulo_GeradorListaInvertida()
  nome_funcao = inspect.currentframe().f_code.co_name # Retorna o nome da função pra passar no logger

  logger.info(f'{nome_modulo} ({nome_funcao}) - Iniciando gerador de lista invertida')

  arquivos_para_lista_invertida = LeArquivo_GLI_CFG(cfgEnum.LEIA)
  documentos = PegaDocumentosDosArquivosXML(arquivos_para_lista_invertida)
  lista_invertida = GeraListaInvertida(documentos)
  SalvaListainvertidaEmCsv(lista_invertida)

  return lista_invertida

In [ ]:
# @title Lendo arquivos e gerando lista invertida

# arquivos_para_lista_invertida = LeArquivo_GLI_CFG(cfgEnum.LEIA)
# documentos = PegaDocumentosDosArquivosXML(arquivos_para_lista_invertida)
# lista_invertida = GeraListaInvertida(documentos)
# #SalvaListainvertidaEmCsv(lista_invertida)

lista_invertida = Modulo_GeradorListaInvertida()

In [ ]:
# @title (DEBUG) - Imprime a lista invertida
#PrintListaInvertida(lista_invertida)

In [ ]:
# @title (DELETAR) Testando a lista invertida
# # o método split só admite um separador , então removemos as vírgulas
# base1 = [
#     ("PN001", "Maria comprou um quilo de banana prata e banana ouro no mercado"),
#     ("PN002", "Um quilo de prata vale muito dinheiro"),
#     ("PN003", "Maria tem dinheiro para comprar um quilo de banana mas não de prata"),
#     ("TF074", ["Quem casa quer casa com quem casou e quem não casou não tem casa"]),
#     ("PN001", "WWWWWWWWWWWWWWWW banana quilo"),
#     ("XS015", ["fumaça preta fumaça laranja", "suco de laranja",
#                "suco de goiaba", "bolo de laranja", "torta de laranja"])
#     ]

# #import gerador_lista_invertida
# #import importlib
# #importlib.reload(gerador_lista_invertida)

# l = GeraListaInvertida(base1)
# SalvaListainvertidaEmCsv(l)
# #PrintListaInvertida(l)

In [ ]:
# @title Lendo os arquivos para gerar a lista

# Indexador

In [ ]:
#%%writefile indexador.py

# Buscador

In [ ]:
#%%writefile buscador.py